## **1994 Wage Inequality Among US Citizens Based on Annual Income**

##### Authors: Danielle Keith, Sam Chin, Jason Zheng

### Introduction

During the 1990s, the United States faced the largest economic expansion it had seen since the 1960s. A huge technological boom brought a 3% economic growth rate, two million new jobs, and a stable 2.1% inflation rate following a minor recession in 1990 (Nasar, 1994). With record low unemployment rates and the need for highly educated employees to keep up with other countries' technological advances, the United States saw an increase in income rates in 1994 compared to previous years (Nasar, 1994). Wage inequality was still prominent for many minorities in the workforce like women, immigrants, and people of colour (Wilson, 2016). 

An analysis of the 1994 census data will allow us to predict whether a given person's income is greater or lesser than/equal to $50,000 based on factors like level of education, age, sex, race, native country, and hours worked per week. 

We will be using the `Adult` dataset taken from https://archive.ics.uci.edu/dataset/2/adult, which has extracted 32,560 entries from the 1994 US census. Of the 32,560 rows, each observation represents a single person and their various attributes. There are 15 columns each signifying a different defining factor. The columns are:

 - `age`: age of individual (integer >0)
 - `workclass`: employment status (ex. self-employed, private, unemployed)
 - `fnlwgt`: final weight or the number of people this individual's entry represents (integer >0)
 - `education`: the highest level of education completed (ex. 12th grade, Bachelor's, Doctorate)
 - `education-num`: the highest level of education completed in numerical form (integer >0)
 - `marital-status`: marital status (ex. married, single)
 - `occupation`: general type of occupation held (ex. sales, services, etc.)
 - `relationship`: primary relationship to others (ex. wife, husband, relative)
 - `race`: racial identity (ex. white, black, asian)
 - `sex`: biological sex (ex. male, female)
 - `capital-gain`: money earned on investments (integer >0)
 - `capital-loss`: money lost on investments (integer >0)
 - `hours-per-week`: hours at work each week
 - `native-country`: country of origin (ex. United States, India, Cuba)
 - `income`: annual income in USD (integer <=50k, >50k)

### Methods

### Results

### Discussion

### References